<a href="https://colab.research.google.com/github/anandchauhan21/pomegranate_disease_prediction/blob/main/pomodetectCreateModel_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load libraries
import numpy as np
import cv2
# from skimage import data
from skimage.color import rgb2hsv
import os
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
# import matplotlib.pyplot as plt
print('Libraries imported...')

Libraries imported...


In [ ]:
# function to detect pomo
def PomoDetect(curImgt):
    # image pre processing
    #cv2.imshow('RGB image', curImgt)
    I = cv2.cvtColor(curImgt, cv2.COLOR_BGR2RGB)  # convert BGR to RGB image
    Igray = cv2.cvtColor(curImgt, cv2.COLOR_BGR2GRAY)  # covert image to grayscale
    Ig = cv2.GaussianBlur(I, (19, 19), 0)  # apply gaussian blur
    # print('Gaussian blurred')
    # Ig = cv2.medianBlur(I, 5)# change this for changing blurring method

    # HSV Conversions
    Ihsv = rgb2hsv(Ig)  # convert gaussian blurred image to HSV
    Ih = np.uint8(255 * Ihsv[:, :, 0])  # hue plane is in range 0-1, change to 0-255 range

    # HSV Filtering for upper hue band
    Hmin = 230
    Hmax = 255
    Ib1 = ((Ih <= Hmax) & (Ih >= Hmin))  # create BW mask within hue range
    # plt.imshow(Ib1, cmap='gray')
    # plt.title('hue mask1')
    # plt.show()

    # HSV Filtering for lower hue band
    Hmin = 0
    Hmax = 30
    Ib2 = ((Ih <= Hmax) & (Ih >= Hmin))  # create BW mask within hue range
    # plt.imshow(Ib2, cmap='gray')
    # plt.title('hue mask2')
    # plt.show()

    # Combining both HSV Filtered mask
    Ib = np.uint8(255 * (Ib1 | Ib2))
    # plt.imshow(Ib, cmap='gray')
    # plt.title('combine hue mask')
    # plt.show()
    # print('HSV Filtering')

    # Filter using contour area and remove small white regions
    cnts = cv2.findContours(Ib, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
        area = cv2.contourArea(c)
        if area < 10000:  # blank all regions less than this pixel area
            cv2.drawContours(Ib, [c], -1, (0, 0, 0), -1)

    # Morph close the inner black regions
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (20, 20))  # create structure element for closing operation
    Iclose = np.uint8(cv2.morphologyEx(Ib, cv2.MORPH_CLOSE, kernel, iterations=2))  # apply morphology closing operation
    # print('Applying contour filtering')
    # print(Iclose[50,:])
    # plt.imshow(Iclose, cmap='gray')
    # plt.title('countour')
    # plt.show()

    # find circles in the BW mask image
    Cmax = 70  # maximum cutt off threshold for circle detection
    NCir = 0  # no of circles assumed 0
    RadMin = 150
    RadMax = 300
    H, W = Iclose.shape
    # print (H,'x', W)
    while NCir == 0 and Cmax > 0:  # run loop till circle detected or Circle Threshold >0
        circles = cv2.HoughCircles(np.uint8(Iclose), cv2.HOUGH_GRADIENT, 2, minDist=100, param1=100,
                                   param2=Cmax, minRadius=RadMin, maxRadius=RadMax)
        # ax = plt.gca()
        if circles is None:  # if no circles detected
            # print('No circles, Cmax=', Cmax)
            NCir = 0
            Cmax = Cmax - 2,  # reduce the Circle Threshold value and repeat the circle detection
            Cmax = Cmax[0]
        else:
            circles2 = np.int16(circles)  # get circles into integer
            NCir = len(circles2[0])  # total circles
            # print('total circles=', NCir)
            # print(circles)
            # for circle in circles2[0, 0]:

            i = circles2[0][0]  # first circle location
            i = np.array(i)
            # tc=circles2[0,:]
            # i=tc[5]
            # print(i)
            # draw the main circle of pomo detected area
            # c1 = plt.Circle((i[0], i[1]), i[2], facecolor='none', edgecolor='b')# draw circle
            # draw the center of the circle
            # c2 = plt.Circle((i[0], i[1]), 2, facecolor='none', edgecolor='b')# draw centre of circle
            Ymin = i[0] - i[2]  # Xmin of circle
            Ymax = i[0] + i[2]  # xmax of circle
            Xmin = i[1] - i[2]  # ymiin of circle
            Xmax = i[1] + i[2]  # ymax of circle
            if Xmin < 0:  # if xmin is less than 0, limit to 0
                Xmin = 0
            if Ymin < 0:
                Ymin = 0
            if Xmax > H:  # if max is > image size limit to H,W
                Xmax = H
            if Ymax > W:
                Ymax = W
            # img = cv2.rectangle(Iclose, (Xmin, Ymin), (Xmax, Ymax), (255, 0, 0))
            # cv2.imshow('Cropped', img)

            # print(Xmin, ':', Xmax, ',', Ymin, ':', Ymax)
            Icut = Igray[Ymin:Ymax, Xmin:Xmax]  # crop the circle image area and return it
            # plt.imshow(Icut, cmap='gray')
            # plt.title('Crop image')
            # plt.show()
            # cv2.imshow('cropped', Icut)
            #        ax.add_artist(c1)
            #        ax.add_artist(c2)

            #      plt.imshow(I, cmap='gray')
            #      plt.title('detected')
            #      plt.show()
    return Icut 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
crnt_path ='/content/drive/'
#ImageDB_Path = os.path.join(crnt_path, 'MyDrive','Colab Notebooks', 'mypomData')# prepares path of google driver
ImageDB_Path='/content/drive/MyDrive/mypomData'
print(ImageDB_Path)

/content/drive/MyDrive/mypomData


In [ ]:
# IMPORTING DATA/IMAGES FROM FOLDERS
RH = 250
RW = 250
count = 0
images = []  # LIST CONTAINING ALL THE IMAGES
classNo = []  # LIST CONTAINING ALL THE CORRESPONDING CLASS ID OF IMAGES
myList = os.listdir(ImageDB_Path)  # print the path
print("Total Classes Detected:", len(myList))  # list the total class (types of pomo, here only 2)
noOfClasses = len(myList)  # length of list
print("Importing Classes .......")


Total Classes Detected: 4
Importing Classes .......


In [ ]:
FolderName = 0
myPicList = os.path.join(ImageDB_Path,str(FolderName))
print(myPicList)

/content/drive/MyDrive/mypomData/0


In [ ]:

for FolderName in range(0, noOfClasses):  # read all folders
    #myPicList = os.listdir(ImageDB_Path + "\\" + str(FolderName))  # read folder
    PicLocations = os.path.join(ImageDB_Path,str(FolderName))
    myPicList = os.listdir(PicLocations)
    print('\n\n *********************Reading folder: ', FolderName, end=", ******************************\n")
    for y in myPicList:  # read all images in a folder
        #tpath = ImageDB_Path + "\\" + str(FolderName) + "\\" + y
        tpath = os.path.join(PicLocations,y)
        print(tpath)
        curImg = cv2.imread(tpath)  # read image
        #cv2.imshow('org file', np.array(curImg))

        Icut = PomoDetect(curImg)  # detect pomo area in that image
        # cv2.waitKey(0)
        Icut2 = Icut / 255  # scale the image
        Ir = cv2.resize(Icut2, (RH, RW))  # resize the image to 125x125
        # curImg = cv2.cvtColor(curImg,cv2.COLOR_BGR2HSV)
        images.append(Ir)  # append to image database
        classNo.append(FolderName)  # append classes to classes database




 *********************Reading folder:  0, ******************************
/content/drive/MyDrive/mypomData/0/img001-00012.jpg
/content/drive/MyDrive/mypomData/0/img001-00014.jpg
/content/drive/MyDrive/mypomData/0/img001-00015.jpg
/content/drive/MyDrive/mypomData/0/img001-00016.jpg
/content/drive/MyDrive/mypomData/0/img001-00017.jpg
/content/drive/MyDrive/mypomData/0/img001-00018.jpg
/content/drive/MyDrive/mypomData/0/img001-00007.jpg
/content/drive/MyDrive/mypomData/0/img001-00008.jpg
/content/drive/MyDrive/mypomData/0/img001-00009.jpg
/content/drive/MyDrive/mypomData/0/img001-00010.jpg
/content/drive/MyDrive/mypomData/0/img001-00011.jpg
/content/drive/MyDrive/mypomData/0/img001-00013.jpg
/content/drive/MyDrive/mypomData/0/img001-00019.jpg
/content/drive/MyDrive/mypomData/0/img001-00001.jpg
/content/drive/MyDrive/mypomData/0/img001-00002.jpg
/content/drive/MyDrive/mypomData/0/img001-00003-not detecting.jpg
/content/drive/MyDrive/mypomData/0/img001-00004.jpg
/content/drive/MyDrive/mypo

In [ ]:
print(" ")
print("Total Images in Images List = ", len(images))
print("Total IDS in classNo List= ", len(classNo))

# CONVERT TO NUMPY ARRAY
images = np.array(images)
classNo = np.array(classNo)
print('image size=' + str(images.shape))

 
Total Images in Images List =  129
Total IDS in classNo List=  129
image size=(129, 250, 250)


In [ ]:
# SPLITTING THE DATA
testRatio = 0.2
valRatio = 0.2
imageDimensions = (RH, RW, 3)  # image dimension for processing
batchSizeVal = 2
epochsVal = 30  # no of epoch for making algorithm best fit
stepsPerEpochVal = 2000
print("Parameters initialised")

Parameters initialised


In [ ]:
# split the training and test images
X_train, X_test, y_train, y_test = train_test_split(images, classNo, test_size=testRatio)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=valRatio)
print('Training Image size=' + str(X_train.shape))
print('Test Image size=' + str(X_test.shape))
print('Validation images size=' + str(X_validation.shape))

X_train = np.array(list(X_train))
X_test = np.array(list(X_test))
X_validation = np.array(list(X_validation))

Training Image size=(82, 250, 250)
Test Image size=(26, 250, 250)
Validation images size=(21, 250, 250)


In [ ]:
# RESHAPE IMAGE DATABASES
print(X_train.shape[0], X_train.shape[1], X_train.shape[2])
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)
X_validation = X_validation.reshape(X_validation.shape[0], X_validation.shape[1], X_validation.shape[2], 1)
print('Image data reshaping done')

# ONE HOT ENCODING OF MATRICES
y_train = to_categorical(y_train, noOfClasses)
y_test = to_categorical(y_test, noOfClasses)
y_validation = to_categorical(y_validation, noOfClasses)

82 250 250
Image data reshaping done


In [ ]:
# CREATING THE MODEL
def myModel():
    noOfFilters = 60  # no of filters to be used
    sizeOfFilter1 = (5, 5)  # first convolution filter size
    sizeOfFilter2 = (3, 3)  # second convolution filter size
    sizeOfPool = (2, 2)
    noOfNodes = 1000  # no of NN nodes

    model = Sequential()  # initialise model type
    model.add((Conv2D(noOfFilters, sizeOfFilter1, input_shape=(imageDimensions[0],
                                                               imageDimensions[1], 1),
                      activation='relu')))  # add layer 1
    model.add((Conv2D(noOfFilters, sizeOfFilter1, activation='relu')))  # layer 2
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add((Conv2D(noOfFilters // 2, sizeOfFilter2, activation='relu')))  # layer 3
    model.add((Conv2D(noOfFilters // 2, sizeOfFilter2, activation='relu')))  # layer 4
    model.add(MaxPooling2D(pool_size=sizeOfPool))
    model.add(Dropout(0.2))

    model.add(Flatten())  # convert 2D feature matrix to 1D vector
    model.add(Dense(noOfNodes, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(noOfClasses, activation='softmax'))

    model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])  # compile the model
    return model

In [ ]:
model = myModel()  # create the model
print('++++++++++++++ MODEL CREATED ++++++++++')
print(model.summary())

++++++++++++++ MODEL CREATED ++++++++++
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 246, 246, 60)      1560      
                                                                 
 conv2d_1 (Conv2D)           (None, 242, 242, 60)      90060     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 121, 121, 60)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 119, 119, 30)      16230     
                                                                 
 conv2d_3 (Conv2D)           (None, 117, 117, 30)      8130      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 58, 58, 30)       0         
 2D)            

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# STARTING THE TRAINING PROCESS
history = model.fit(X_train, y_train, epochs=epochsVal)
print('+++++++++ MODEL FITTING DONE ++++++++++++')

Epoch 1/30
3/3 [==============================] - 83s 25s/step - loss: 2.8938 - accuracy: 0.3780
Epoch 2/30
3/3 [==============================] - 78s 24s/step - loss: 1.5903 - accuracy: 0.2683
Epoch 3/30
3/3 [==============================] - 78s 24s/step - loss: 1.0395 - accuracy: 0.4390
Epoch 4/30
3/3 [==============================] - 78s 24s/step - loss: 0.7614 - accuracy: 0.6951
Epoch 5/30
3/3 [==============================] - 78s 24s/step - loss: 0.6626 - accuracy: 0.7683
Epoch 6/30
3/3 [==============================] - 79s 24s/step - loss: 0.5823 - accuracy: 0.8171
Epoch 7/30
3/3 [==============================] - 79s 24s/step - loss: 0.5029 - accuracy: 0.8293
Epoch 8/30
3/3 [==============================] - 79s 24s/step - loss: 0.4468 - accuracy: 0.8293
Epoch 9/30
3/3 [==============================] - 82s 25s/step - loss: 0.3695 - accuracy: 0.8902
Epoch 10/30
3/3 [==============================] - 79s 24s/step - loss: 0.3125 - accuracy: 0.9024
Epoch 11/30
3/3 [============

In [ ]:
# save model
crnt_path ='/content/drive/'
FLoc= os.path.join(crnt_path,'MyDrive','Colab Notebooks','model.h5')
print('Model file location', FLoc)
model.save(FLoc)
print('Creted model saved as, model.h5')

Model file location /content/drive/MyDrive/Colab Notebooks/model.h5
Creted model saved as, model.h5
